In [1]:
! pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
import shutil
from pyspark.sql.types import *
import pyspark.sql.functions as Spark_f
from pyspark import SparkConf

In [3]:
conf = SparkConf() \
    .setMaster("local[10]") \
    .set("spark.driver.memory", "2g") \
    .set("spark.executor.memory", "4g") \
    .set("spark.shuffle.spill", "true") \
    .set("spark.network.timeout", "60s") \
    .set("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:G1ReservePercent=20")

spark = SparkSession.builder.config(conf=conf).appName("PySparkGameSalesJob").getOrCreate()

24/10/21 15:03:12 WARN Utils: Your hostname, NBM-WXX9-7572c0be resolves to a loopback address: 127.0.1.1; using 192.168.88.79 instead (on interface wlp1s0)
24/10/21 15:03:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/21 15:03:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
# image_struct = StructType([
#     StructField("small_image_url", StringType(), True),
#     StructField("medium_image_url", StringType(), True),
#     StructField("large_image_url", StringType(), True),
#     StructField("attachment_type", StringType(), True)
# ])
# book_review_struct = StructType([StructField("rating", FloatType(), True), StructField("title", StringType(), True), StructField("text", StringType(), True), StructField("images", ArrayType(ArrayType(image_struct, True), True), True), StructField(
#     "asin", StringType(), True), StructField("parent_asin", StringType(), True), StructField("user_id", StringType(), True), StructField("timestamp", IntegerType(), True), StructField("verified_purchase", BooleanType(), True), StructField("helpful_vote", IntegerType(), True)])
book_review_struct = book_review_struct = StructType([
    StructField("rating", FloatType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("images", ArrayType(MapType(StringType(), StringType(), True), True), True),
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("verified_purchase", BooleanType(), True),
    StructField("helpful_vote", IntegerType(), True)
])
# book_review_struct = StructType([
#     StructField("rating", FloatType(), True),
#     StructField("title", StringType(), True),
#     StructField("text", StringType(), True),
#     StructField("images", ArrayType(
#         StructType([
#             StructField("small_image_url", StringType(), True),
#             StructField("medium_image_url", StringType(), True),
#             StructField("large_image_url", StringType(), True),
#             StructField("attachment_type", StringType(), True)
#         ])
#     ), True),
#     StructField("asin", StringType(), True),
#     StructField("parent_asin", StringType(), True),
#     StructField("user_id", StringType(), True),
#     StructField("timestamp", LongType(), True),
#     StructField("verified_purchase", BooleanType(), True),
#     StructField("helpful_vote", IntegerType(), True)
# ])
book_review = spark.read.schema(book_review_struct).json("/dataset/Books.jsonl.gz")
book_review.show()

+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----------------+------------+
|rating|               title|                text|              images|      asin|parent_asin|             user_id|    timestamp|verified_purchase|helpful_vote|
+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----------------+------------+
|   1.0|Not a watercolor ...|It is definitely ...|[{small_image_url...|B09BGPFTDB| B09BGPFTDB|AFKZENTNBQ7A7V7UX...|1642399598485|             true|           0|
|   5.0|Updated: after 1s...|Updated: after fi...|                  []|0593235657| 0593235657|AFKZENTNBQ7A7V7UX...|1640629604904|             true|           1|
|   5.0|Excellent! I love...|I bought it for t...|                  []|1782490671| 1782490671|AFKZENTNBQ7A7V7UX...|1640383495102|             true|           0|
|   5.0|Updated after 1st...|Updat

In [6]:
print(len(book_review.columns))
book_review.select("images").show(20, False)

10
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
book_meta_struct = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", FloatType(), True),
    StructField("rating_number", IntegerType(), True),
    StructField("features", ArrayType(StringType(),True), True),
    StructField("description", ArrayType(StringType(),True), True),
    StructField("price", FloatType(), True),
    StructField("images", ArrayType(MapType(StringType(), StringType(), True), True), True),
    StructField("videos", ArrayType(MapType(StringType(), StringType(), True), True), True),
    StructField("store", StringType(), True),
    StructField("categories", ArrayType(StringType(),True), True),
    StructField("details", MapType(StringType(), StringType(), True), True),
    StructField("parent_asin", StringType(), True),
    StructField("bought_together", StringType(), True)
])
book_meta = spark.read.schema(book_meta_struct).json("/dataset/meta_Books.jsonl.gz")
book_meta.show()

+-------------+--------------------+--------------+-------------+--------------------+--------------------+-----+--------------------+------+--------------------+--------------------+--------------------+-----------+---------------+
|main_category|               title|average_rating|rating_number|            features|         description|price|              images|videos|               store|          categories|             details|parent_asin|bought_together|
+-------------+--------------------+--------------+-------------+--------------------+--------------------+-----+--------------------+------+--------------------+--------------------+--------------------+-----------+---------------+
|        Books|             Chaucer|           4.5|           29|                  []|                  []| 8.23|[{large -> https:...|    []|Peter Ackroyd (Au...|[Books, Literatur...|{Publisher -> Cha...| 0701169850|           NULL|
|        Books|Notes from a Kidw...|           5.0|            1|[Co

In [8]:
# book_meta.filter(book_meta.videos!=Spark_f.array()).show(20, False)
# book_meta.filter(Spark_f.size(book_meta.videos)>=1).select("videos").show(20, False)

In [9]:
book_meta = book_meta.drop(book_meta.bought_together, book_meta.store, book_meta.price) # пустой столбец, какой магазин, цена

In [10]:
# book_meta.withColumn("num_categories", Spark_f.size(book_meta.categories)).groupBy("num_categories").count().show()

In [11]:
book_meta = book_meta.filter(Spark_f.size(book_meta.categories)>0)

In [12]:
# book_meta.groupBy("main_category").count().sort(Spark_f.desc("count")).show(40,False)

In [13]:
book_meta = book_meta[(book_meta.main_category=="Books") | (book_meta.main_category=="Buy a Kindle") | (book_meta.main_category=="Audible Audiobooks")]

In [14]:
book_meta = book_meta.withColumn("first_category", Spark_f.col("categories")[0]) \
       .withColumn("second_category", Spark_f.col("categories")[1]) \
       .withColumn("remaining_categories", Spark_f.expr("slice(categories, 3, size(categories)-2)")).drop(book_meta.categories)

In [15]:
# book_meta.select("first_category", "second_category", "remaining_categories").show(40,False)

In [16]:
# book_review.groupBy("verified_purchase").count().show()

In [17]:
book_review.columns

['rating',
 'title',
 'text',
 'images',
 'asin',
 'parent_asin',
 'user_id',
 'timestamp',
 'verified_purchase',
 'helpful_vote']

In [18]:
book_review = book_review.drop(book_review.asin, book_review.images)
book_review = book_review.filter(book_review.verified_purchase == True)

In [19]:
# book_meta.groupBy("title").count().sort(Spark_f.desc("count")).show()

In [20]:
# book_review.groupBy("user_id").count().select(Spark_f.avg("count")).show()
# book_review.groupBy("user_id").count().sort(Spark_f.desc("count")).show(20, False)

In [21]:
from pyspark import StorageLevel
# print(book_meta.count())
# book_meta = book_meta.join(book_review, "parent_asin", "leftsemi").persist(StorageLevel.MEMORY_AND_DISK)
book_meta = book_meta.join(book_review, "parent_asin", "leftsemi")
# print(book_meta.count())

In [22]:
# print(book_review.count())
# book_review = book_review.join(book_meta, "parent_asin", "leftsemi").persist(StorageLevel.MEMORY_AND_DISK)
book_review = book_review.join(book_meta, "parent_asin", "leftsemi")
# rows_reviews = book_review.count()
# print(rows_reviews)

In [23]:
rows_reviews = 20_000_000 # около 20 миллионов подтверждённых отзывов, чтобы лишний раз не считать
rows_reviews = int(rows_reviews*0.01)
best_reviews = book_review.orderBy(col("helpful_vote").desc()).limit(rows_reviews)
# best_reviews.show(20, False)

In [ ]:
# best_reviews.coalesce(1).write.mode("overwrite") \
#     .option("compression", "snappy") \
#     .parquet("/dataset/output/best_reviews")
best_reviews.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/output/best_reviews")

In [ ]:
# book_review.coalesce(1).write.mode("overwrite") \
#     .option("compression", "snappy") \
#     .parquet("/dataset/output/book_reviews")
book_review.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/output/book_reviews")

In [26]:
# book_review.coalesce(1).write.mode("overwrite") \
#     .option("compression", "snappy") \
#     .parquet("/dataset/output/book_metadata")
book_meta.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/output/book_metadata")

24/10/21 15:03:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [27]:
# book_review.unpersist()
# book_meta.unpersist()